<center>

<h1 style="text-align:center"> Streams, Laziness and Memoization </h1>
<h2 style="text-align:center"> CS3100 Monsoon 2020 </h2>
</center>


$
\require{color}
\newcommand{\colorred}[1]{\color{\red}{\text{#1}}}
$


## Review

### Previously

* Modular Programming
  + Namespacing, Abstraction, Code Reuse
  + Structures, Signatures, Functors
  
### This lecture

* Streams: Programming with infinite data structures
* Laziness: Call-by-need evaluation

## Recursive values

* In OCaml, we can define recursive functions.
  + we can also define **recursive values**

In [ ]:
(* Infinite list of ones *)
let rec ones = 1::ones

In [ ]:
(* Infinite list of alternating 0s and 1s *)
let rec zero_ones = 0::1::zero_ones

Even though the list is **infinite**, the data structure uses **finite** memory. 

## Infinite data structures

Infinite data structures are not just an intellectual curiosity.

* Infinite sequences such as primes and fibonacci numbers.
* Streams of input read from file or socket.
* Game trees which may be infinite
  + Every possible move leads to branch in the tree. 
  + Imagine game trees where a piece could chase the other around forever. 

## Limitations of cyclic structures

Suppose we want to convert the infinite list `zero_ones` to string, the obvious solutions don't work. 

In [ ]:
let zero_ones_string = List.map string_of_int zero_ones

## List to Streams

Start off from what we know -- list datatype:

```ocaml
type 'a list = Nil | Cons of 'a * 'a list
```

and make a **stream** type.

In [ ]:
type 'a stream = Cons of 'a * 'a stream

There is no `Nil` since the streams are infinite. 

## Doesn't quite work

In [ ]:
let rec zero_ones = Cons (0, Cons (1, zero_ones))

In [ ]:
let rec to_string (Cons(x,xs)) = Cons(string_of_int x, to_string xs)

In [ ]:
to_string zero_ones

## Pausing the execution

* We need a way to pause the execution at the tail position
    + rather than recursively applying to the rest of the list. 
* Use **thunks**: `unit -> 'a` functions.

## Pausing the execution

In [ ]:
let v = failwith "error"

In [ ]:
let f = fun () -> failwith "error"

In [ ]:
f ()

## Streams again

Use a thunk for the tail to pause execution

In [ ]:
type 'a stream = Cons of 'a * (unit -> 'a stream)

In [ ]:
let rec zero_ones = Cons (0, fun () -> Cons (1, fun () -> zero_ones))

In [ ]:
let hd (Cons (x, _)) = x

In [ ]:
let tl (Cons (_, xs)) = xs ()

## More Stream functions

In [ ]:
(* [take n s] returns a list with the first [n] elements of the stream [s] *)
let rec take n s = 
  if n = 0 then []
  else (hd s)::(take (n-1) (tl s))

In [ ]:
take 10 zero_ones

In [ ]:
(* [drop n s] returns a new stream with the first [n] elements of stream [s] dropped *)
let rec drop n s =
  if n = 0 then s
  else drop (n-1) (tl s)

In [ ]:
drop 1 zero_ones (* [1;0;1;0;...] *)

## Higher order functions on streams

In [ ]:
let rec map f s = Cons (f (hd s), fun () -> map f (tl s))

In [ ]:
let zero_ones_str = map string_of_int zero_ones

In [ ]:
take 10 zero_ones_str

## Higher order functions on streams

In [ ]:
(** [filter p s] returns a new stream where every element [x] from [s] 
    such that [p x = true] is removed *)
let rec filter p s =
  if p (hd s) then filter p (tl s)
  else Cons (hd s, fun () -> filter p (tl s))

In [ ]:
let s' = filter ((=) 0) zero_ones in
take 10 s'

## Higher order functions on streams

In [ ]:
let rec zip f s1 s2 = Cons (f (hd s1) (hd s2), fun () -> zip f (tl s1) (tl s2))

In [ ]:
zip (fun x y -> (x,y)) zero_ones zero_ones_str

## Higher order functions on streams

In [ ]:
(*
[0;1;0;1;....]
+
[1;0;1;0;....]
*)

let s = zip (+) zero_ones (drop 1 zero_ones)

In [ ]:
take 10 s

## Primes

* **Sieve of Eratosthenes**: Neat way to compute primes.
* Start with a stream `s` of `[2;3;4;.....]`.
* At each step, 
  + `p = hd s` is a prime.
  + return a new stream `s'` such that $\forall x \in s.x \text{ mod } p = 0 \notin s'$

## Primes

* In the first step,
  + `prime = 2`
  + `new stream = [3;5;7;9;11;13;15;17;....]`
* In the second step,
  + `prime = 3`
  + `new stream = [5;7;11;13;17;19;23;....]`

## Primes

In [ ]:
(* [from n] returns a natural number stream from [n] *)
let rec from n = Cons (n, fun () -> from (n+1));;
from 2

In [ ]:
let rec primes s = 
  let p = hd s in
  Cons (p, fun () -> primes @@ filter (fun x -> x mod p = 0) (tl s))

let primes_stream = primes (from 2)

In [ ]:
take 100 primes_stream

## Fibonacci sequence

* Let's consider Fibonacci sequence
  + `s1 = [1;1;2;3;5;8;13;...]`
* Let's consider the tail of `s1`
  + `s2 = [1;2;3;5;8;13;....]`
* Let's zip `s1` and `s2` by adding together the elements:
  + `s3 = [2;3;5;8;13;21;...]`
  + `s3` is nothing but the tail of tail of fibonacci sequence. 
* If we were to prepend `[1;1]` to `s3` we will have the fibonacci sequence.



## Fibonacci sequence

In [ ]:
let rec fibs = 
  Cons (1, fun () -> 
    Cons (1, fun () -> 
      zip (+) fibs (tl fibs)))

In [ ]:
take 30 fibs

## Fibonacci sequence : Efficiency

* Each time we force the computation of the next element, we compute the fibonacci of previous element twice.
  + Not immediately apparent, but this is equivalent to:
  
```ocaml
let rec fib n = 
  if n < 2 then 1 
  else fib (n-1) + fib (n-2)
```
* There is an exponential increase in the running time of `fib(n)` for each increase in `n`.

## Lazy Values

* It would be nice to **save** the results of the execution for previously seen values and reuse them.
  + This is the idea behind lazy values in OCaml.
* Lazy values are the opt-in, explicit, **call-by-need** reduction strategy for OCaml
  + Rest of the language is strict i.e, call-by-value

## Lazy Values

* Lazy module in OCaml is:

```ocaml
module Lazy = struct
  type 'a t = 'a lazy_t
  val force : 'a t -> 'a
end
```

OCaml has syntactic support for lazy values through the `lazy` keyword.

## Lazy values

In [ ]:
let v = lazy (10 + (print_endline "Hello"; 20))

In [ ]:
Lazy.force v

In [ ]:
Lazy.force v

## Time it

Let us define a helper function for timing a function execution.

In [5]:
let time_it f =
  let t = Unix.gettimeofday () in
  let r = f () in
  (r, (Unix.gettimeofday() -. t) *. 1000.)

val time_it : (unit -> 'a) -> 'a * float = <fun>


## Lazy fib

In [ ]:
(* 30th Fibonacci number *)
let fib30lazy = lazy (take 30 fibs |> List.rev |> List.hd)

In [ ]:
let (r,d) = time_it (fun () -> Lazy.force fib30lazy) in
Printf.printf "fib(30) = %d, time = %f milliseconds\n%!" r d

## Lazy fib

In [ ]:
let fib31lazy = lazy (take 31 fibs |> List.rev |> List.hd)

In [ ]:
let (r,d) = time_it (fun () -> Lazy.force fib31lazy) in
Printf.printf "fib(31) = %d, time = %f milliseconds\n%!" r d

We're not reusing `fib(30)` in `fib(31)` :-(

## Lazy stream

Let's redefine the stream using lazy values (instead of thunks)

In [ ]:
type 'a stream = Cons of 'a * 'a stream Lazy.t

In [ ]:
let hd (Cons (x,l)) = x
let tl (Cons (x,l)) = Lazy.force l
let rec take n s = 
  if n = 0 then [] else (hd s)::(take (n-1) (tl s))
let rec zip f s1 s2 = 
  Cons (f (hd s1) (hd s2), lazy (zip f (tl s1) (tl s2)))

## Fibs Lazy Streams

In [ ]:
let rec fibslazystream = 
  Cons (1, lazy (
    Cons (1, lazy (
      zip (+) fibslazystream (tl fibslazystream)))))

In [ ]:
let f () = take 100 fibslazystream |> List.rev |> List.hd in
let (r,d) = time_it f in
Printf.printf "fib(n) = %d, time = %f milliseconds\n%!" r d

You can see that this is fast!

## Memoization

* Lazy values in OCaml are a specific efficient implementation of the general idea of caching called **Memoization**.
  + Add caching to functions to retrieve results fast. 

In [4]:
let memo f = 
  let cache = Hashtbl.create 16 in
  fun v -> 
    match Hashtbl.find_opt cache v with
    | None -> 
        let res = f v in 
        Hashtbl.add cache v res;
        res
    | Some res -> res

val memo : ('a -> 'b) -> 'a -> 'b = <fun>


## Expensive identity

In [ ]:
let rec spin n = if n = 0 then () else spin (n-1)

In [ ]:
let expensive_id x = spin 200000000; x

In [ ]:
let (r,d) = time_it (fun () -> expensive_id 10) in
Printf.printf "expensive_id(n) = %d, time = %f milliseconds\n%!" r d

## Memoizing expensive identity

In [ ]:
let memoized_expensive_id = memo expensive_id

In [ ]:
let (r,d) = time_it (fun () -> memoized_expensive_id 10) in
Printf.printf "memoized_expensive_id(n) = %d, time = %f milliseconds\n%!" r d

## Memoizing recursive functions

* Memoizing recursive functions is a bit more tricky.
  + We need to tie the **recursive knot**

In [ ]:
let rec fib n = 
  if n < 2 then 1 else fib(n-2) + fib(n-1)

In [ ]:
let (r,d) = time_it (fun () -> fib 40) in
Printf.printf "fib(n) = %d, time = %f milliseconds\n%!" r d

## Memoizing recursive functions

Simply doing `let memo_fib = memo fib` will only memoize the outer calls and not the recursive calls.

In [ ]:
let memo_fib = memo fib

In [ ]:
let (r,d) = time_it (fun () -> memo_fib 40) in
Printf.printf "memo_fib(n) = %d, time = %f milliseconds\n%!" r d

In [ ]:
let (r,d) = time_it (fun () -> memo_fib 39) in
Printf.printf "memo_fib(n) = %d, time = %f milliseconds\n%!" r d

## Tying the recursive knot

This function should remind you of the definition we used for Y combinator.

In [ ]:
let fib_norec f n = if n < 2 then 1 else f (n-1) + f (n-2)

The idea is to provide an `f` which is the memoized version of the usual recursive `fib` function:

```ocaml
let rec fib n = if n < 2 then 1 else fib (n-1) + fib (n-2)
```

We will use a **reference** to tie the knot.

## Tying the recursive knot

`memo_rec` will memoize recursive function that take an explicit recursive function argument such as `fib_norec`.

In [11]:
let memo_rec f_norec =
  (* 1: Reference [f_ref] to a dummy function *)
  let f_ref = ref (fun _ -> assert false) in
  (* 2: memoize the "eta-expanded" [f_norec] function by dereferencing [f]. *)
  let f_rec_memo = memo (fun x -> f_norec !f_ref x) in
                                          (* not dereferencing [!f_ref] yet *)
  (* 3: update [f_ref] to the recursive memoized function *)
  f_ref := f_rec_memo;

  f_rec_memo

val memo_rec : (('a -> 'b) -> 'a -> 'b) -> 'a -> 'b = <fun>


<center>
    
<img src="rec_knot.svg">
</center>

## Memoized fib

In [ ]:
let fib_memo = memo_rec fib_norec

In [ ]:
let r,d = time_it (fun () -> fib_memo 40) in
Printf.printf "memo_fib(n) = %d, time = %f milliseconds\n%!" r d

In [ ]:
let r,d = time_it (fun () -> fib_memo 41) in
Printf.printf "memo_fib(n) = %d, time = %f milliseconds\n%!" r d

## Edit distance

* Memoization is a general solution for **dynamic programming**.
* Let's compute **edit distance** (aka **Levenshtein distance**) between two strings.
* Example: 
   * edit_distance("kitten","sitting") = 3
   * $\colorred{k}$itten -> $\colorred{s}$itten
   * sitt$\colorred{e}$n -> sitt$\colorred{i}$n
   * sittin -> sittin$\colorred{g}$

## Edit distance

In [6]:
(* See Real World OCaml book : https://dev.realworldocaml.org/imperative-programming.html#scrollNav-5 *)
let rec edit_distance ?log (s,t) = 
  let open String in
  if log = Some true then print_endline (s ^ " " ^ t);
  match String.length s, String.length t with
  | 0,x | x,0 -> x
  | len_s, len_t ->
    let s' = sub s 0 (len_s - 1) in
    let t' = sub t 0 (len_t - 1) in
    (* Minimum of three possibilities *)
    List.fold_left (fun acc v -> min acc v) max_int [
      edit_distance ?log (s',t) + 1; (* insert at end of s *)
      edit_distance ?log (s,t') + 1; (* delete from end of s *)
      edit_distance ?log (s',t') +   
        if get s (len_s-1) = get t (len_t-1) then 0 (* no change *)
        else 1 (* replace *)
    ]

val edit_distance : ?log:bool -> string * string -> int = <fun>


## Edit distance

In [7]:
time_it (fun () -> (edit_distance ~log:true) ("OCaml", "ocaml"))

OCaml ocaml
OCam ocam
OCa oca
OC oc
O o
 
O 
 o
OC o
O 
OC 
O o
 
O 
 o
O oc
 o
O o
 
O 
 o
 oc
OCa oc
OC o
O 
OC 
O o
 
O 
 o
OCa o
OC 
OCa 
OC o
O 
OC 
O o
 
O 
 o
OC oc
O o
 
O 
 o
OC o
O 
OC 
O o
 
O 
 o
O oc
 o
O o
 
O 
 o
 oc
OC oca
O oc
 o
O o
 
O 
 o
 oc
OC oc
O o
 
O 
 o
OC o
O 
OC 
O o
 
O 
 o
O oc
 o
O o
 
O 
 o
 oc
O oca
 oc
O oc
 o
O o
 
O 
 o
 oc
 oca
OCam oca
OCa oc
OC o
O 
OC 
O o
 
O 
 o
OCa o
OC 
OCa 
OC o
O 
OC 
O o
 
O 
 o
OC oc
O o
 
O 
 o
OC o
O 
OC 
O o
 
O 
 o
O oc
 o
O o
 
O 
 o
 oc
OCam oc
OCa o
OC 
OCa 
OC o
O 
OC 
O o
 
O 
 o
OCam o
OCa 
OCam 
OCa o
OC 
OCa 
OC o
O 
OC 
O o
 
O 
 o
OCa oc
OC o
O 
OC 
O o
 
O 
 o
OCa o
OC 
OCa 
OC o
O 
OC 
O o
 
O 
 o
OC oc
O o
 
O 
 o
OC o
O 
OC 
O o
 
O 
 o
O oc
 o
O o
 
O 
 o
 oc
OCa oca
OC oc
O o
 
O 
 o
OC o
O 
OC 
O o
 
O 
 o
O oc
 o
O o
 
O 
 o
 oc
OCa oc
OC o
O 
OC 
O o
 
O 
 o
OCa o
OC 
OCa 
OC o
O 
OC 
O o
 
O 
 o
OC oc
O o
 
O 
 o
OC o
O 
OC 
O o
 
O 
 o
O oc
 o
O o
 
O 
 o
 oc
OC oca
O oc
 o
O o
 
O 
 o
 oc
OC oc


O oc
 o
O o
 
O 
 o
 oc
O oca
 oc
O oc
 o
O o
 
O 
 o
 oc
 oca
O ocam
 oca
O oca
 oc
O oc
 o
O o
 
O 
 o
 oc
 oca
 ocam
OCa ocaml
OC ocam
O oca
 oc
O oc
 o
O o
 
O 
 o
 oc
 oca
OC oca
O oc
 o
O o
 
O 
 o
 oc
OC oc
O o
 
O 
 o
OC o
O 
OC 
O o
 
O 
 o
O oc
 o
O o
 
O 
 o
 oc
O oca
 oc
O oc
 o
O o
 
O 
 o
 oc
 oca
O ocam
 oca
O oca
 oc
O oc
 o
O o
 
O 
 o
 oc
 oca
 ocam
OCa ocam
OC oca
O oc
 o
O o
 
O 
 o
 oc
OC oc
O o
 
O 
 o
OC o
O 
OC 
O o
 
O 
 o
O oc
 o
O o
 
O 
 o
 oc
O oca
 oc
O oc
 o
O o
 
O 
 o
 oc
 oca
OCa oca
OC oc
O o
 
O 
 o
OC o
O 
OC 
O o
 
O 
 o
O oc
 o
O o
 
O 
 o
 oc
OCa oc
OC o
O 
OC 
O o
 
O 
 o
OCa o
OC 
OCa 
OC o
O 
OC 
O o
 
O 
 o
OC oc
O o
 
O 
 o
OC o
O 
OC 
O o
 
O 
 o
O oc
 o
O o
 
O 
 o
 oc
OC oca
O oc
 o
O o
 
O 
 o
 oc
OC oc
O o
 
O 
 o
OC o
O 
OC 
O o
 
O 
 o
O oc
 o
O o
 
O 
 o
 oc
O oca
 oc
O oc
 o
O o
 
O 
 o
 oc
 oca
OC ocam
O oca
 oc
O oc
 o
O o
 
O 
 o
 oc
 oca
OC oca
O oc
 o
O o
 
O 
 o
 oc
OC oc
O o
 
O 
 o
OC o
O 
OC 
O o
 
O 
 o
O oc
 o
O o
 
O 
 o

- : int * float = (2, 18.3479785919189453)


## Edit distance

In [8]:
time_it (fun () -> (edit_distance ~log:false) ("OCaml 4.08", "ocaml 4.08"))

- : int * float = (2, 6670.32909393310547)


## Memoize edit distance

In [12]:
let rec edit_distance_norec ?log f (s,t) = 
  let open String in
  if log = Some true then print_endline (s ^ " " ^ t);
  match String.length s, String.length t with
  | 0,x | x,0 -> x
  | len_s, len_t ->
    let s' = sub s 0 (len_s - 1) in
    let t' = sub t 0 (len_t - 1) in
    List.fold_left (fun acc v -> min acc v) max_int [
      f (s',t) + 1; (* insert at end of s *)
      f (s,t') + 1; (* delete from end of s *)
      f (s',t') +   
        if get s (len_s-1) = get t (len_t-1) then 0 (* no change *)
        else 1 (* replace *)
    ]

val edit_distance_norec :
  ?log:bool -> (string * string -> int) -> string * string -> int = <fun>


## Memoize edit distance

In [13]:
let memo_edit_distance = memo_rec (edit_distance_norec ~log:true)

val memo_edit_distance : string * string -> int = <fun>


In [14]:
time_it (fun () -> memo_edit_distance ("OCaml 4.08", "ocaml 4.08"))

OCaml 4.08 ocaml 4.08
OCaml 4.0 ocaml 4.0
OCaml 4. ocaml 4.
OCaml 4 ocaml 4
OCaml  ocaml 
OCaml ocaml
OCam ocam
OCa oca
OC oc
O o
 
O 
 o
OC o
OC 
O oc
 oc
OCa oc
OCa o
OCa 
OC oca
O oca
 oca
OCam oca
OCam oc
OCam o
OCam 
OCa ocam
OC ocam
O ocam
 ocam
OCaml ocam
OCaml oca
OCaml oc
OCaml o
OCaml 
OCam ocaml
OCa ocaml
OC ocaml
O ocaml
 ocaml
OCaml  ocaml
OCaml  ocam
OCaml  oca
OCaml  oc
OCaml  o
OCaml  
OCaml ocaml 
OCam ocaml 
OCa ocaml 
OC ocaml 
O ocaml 
 ocaml 
OCaml 4 ocaml 
OCaml 4 ocaml
OCaml 4 ocam
OCaml 4 oca
OCaml 4 oc
OCaml 4 o
OCaml 4 
OCaml  ocaml 4
OCaml ocaml 4
OCam ocaml 4
OCa ocaml 4
OC ocaml 4
O ocaml 4
 ocaml 4
OCaml 4. ocaml 4
OCaml 4. ocaml 
OCaml 4. ocaml
OCaml 4. ocam
OCaml 4. oca
OCaml 4. oc
OCaml 4. o
OCaml 4. 
OCaml 4 ocaml 4.
OCaml  ocaml 4.
OCaml ocaml 4.
OCam ocaml 4.
OCa ocaml 4.
OC ocaml 4.
O ocaml 4.
 ocaml 4.
OCaml 4.0 ocaml 4.
OCaml 4.0 ocaml 4
OCaml 4.0 ocaml 
OCaml 4.0 ocaml
OCaml 4.0 ocam
OCaml 4.0 oca
OCaml 4.0 oc
OCaml 4.0 o
OCaml 4.0 
OCaml 4. ocam

- : int * float = (2, 0.912189483642578125)


## Summary

* Laziness and Memoization make sense only in a **pure** setting
  + If your programs contains side-effects, laziness and memoization will not duplicate it.
* OCaml doesn't prevent you from performing side-effects in your program
  + Same as C, C++, Java, Python, Rust, Swift, JavaScript, etc. 
  + Haskell tracks side-effects in the type system and compartmentalizes pure from side-effecting code. 
* Using purity in your program helps you take advantage of lazy evalution, memoization, etc, in **any** lanugage. 

<center>

<h1 style="text-align:center"> Fin. </h1>
</center>